In [1]:
import pandas as pd

In [2]:
import requests
from urllib.parse import urlencode

In [3]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
data_1 = 'https://disk.yandex.ru/d/p_UjGG_Dv8CE5Q'
df_final_url = base_url + urlencode(dict(public_key=data_1))
response = requests.get(df_final_url)
df_download_url = response.json()['href']
df = pd.read_csv(df_download_url)

In [4]:
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [5]:
# Преобразование столбца "date" в тип данных datetime
df['date'] = pd.to_datetime(df['date'])

# Извлечение информации о месяце
df['month'] = df['date'].dt.month
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,1
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,1
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,1
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,1


In [6]:
df_mau = df.query("month == '2' and event == 'app_start'").device_id.nunique()

In [7]:
df_install_january = df.query("month == '1' and event == 'app_install'").groupby('event').agg({'device_id':'count'})

In [8]:
df_2 = df[['date', 'event', 'device_id']]

In [9]:
df_install = df_2.query("event == 'app_install'").rename(columns={"date": "date_install"}).drop(columns='event')
df_install.head()

,date_install,device_id
2515061,2020-01-01,4921563
2515062,2020-01-01,1311583
2515063,2020-01-01,23006325
2515064,2020-01-01,5645652
2515065,2020-01-01,8798567


In [10]:
df_purchase = df_2.query("event == 'purchase'").groupby('device_id', as_index=False).agg({'date':'min'}).rename(columns={"date": "date_purchase"})
df_purchase

,device_id,date_purchase
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [11]:
a = df_install.merge(df_purchase, how = 'left', on='device_id')
a

,date_install,device_id,date_purchase
0,2020-01-01,4921563,2020-01-10
1,2020-01-01,1311583,NaT
2,2020-01-01,23006325,2020-01-01
3,2020-01-01,5645652,NaT
4,2020-01-01,8798567,NaT
...,...,...,...
154592,2020-03-31,19717968,NaT
154593,2020-03-31,32966665,NaT
154594,2020-03-31,6335964,NaT
154595,2020-03-31,29155826,NaT


In [12]:
a['diff'] = a.date_purchase - a.date_install
a

,date_install,device_id,date_purchase,diff
0,2020-01-01,4921563,2020-01-10,9 days
1,2020-01-01,1311583,NaT,NaT
2,2020-01-01,23006325,2020-01-01,0 days
3,2020-01-01,5645652,NaT,NaT
4,2020-01-01,8798567,NaT,NaT
...,...,...,...,...
154592,2020-03-31,19717968,NaT,NaT
154593,2020-03-31,32966665,NaT,NaT
154594,2020-03-31,6335964,NaT,NaT
154595,2020-03-31,29155826,NaT,NaT


In [13]:
cogort_7_days = a.query("diff <= '7d' or diff == 'NaT'")
cogort_7_days

,date_install,device_id,date_purchase,diff
1,2020-01-01,1311583,NaT,NaT
2,2020-01-01,23006325,2020-01-01,0 days
3,2020-01-01,5645652,NaT,NaT
4,2020-01-01,8798567,NaT,NaT
5,2020-01-01,8423608,2020-01-06,5 days
...,...,...,...,...
154592,2020-03-31,19717968,NaT,NaT
154593,2020-03-31,32966665,NaT,NaT
154594,2020-03-31,6335964,NaT,NaT
154595,2020-03-31,29155826,NaT,NaT


In [14]:
x = df.query("event == 'app_install'") \
      .groupby('date', as_index = False) \
      .agg({'device_id':'count'}) \
    .rename(columns={"date": "date_install"}) \
      .rename(columns={"device_id": "install_count"})

In [15]:
y = cogort_7_days.groupby('date_install', as_index=False) \
                 .agg({'date_purchase':'count'}) \
                 .rename(columns={"date_purchase": "purchase_count"})

In [16]:
df_f = x.merge(y, how = 'inner', on='date_install')

df_f['CR'] = df_f.purchase_count / df_f.install_count * 100

In [17]:
df_f.sort_values('CR', ascending=False).round(1).head()

,date_install,install_count,purchase_count,CR
0,2020-01-01,3579,1408,39.3
8,2020-01-09,1424,558,39.2
14,2020-01-15,4310,1650,38.3
13,2020-01-14,5173,1973,38.1
1,2020-01-02,3144,1186,37.7


In [18]:
df_utm_source = df.query("event == 'app_install'") \
      .groupby('utm_source', as_index = False) \
      .agg({'device_id':'count'}).sort_values('device_id')

In [19]:
df_register = df_2.query("event == 'register'").rename(columns={"date": "date_register"}).drop(columns='event')
df_register.head()

,date_register,device_id
2669658,2020-01-01,294193
2669659,2020-01-01,22917617
2669660,2020-01-01,15248490
2669661,2020-01-01,252062
2669662,2020-01-01,2251583


In [20]:
dff = df.merge(df_register, how = 'inner', on='device_id')
dff.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,date_register
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1,2020-01-07
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,1,2020-01-07
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,1,2020-01-07
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,1,2020-01-07
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,1,2020-01-07


In [21]:
#дата регистрации < даты совершения события, то пользователь уже зарегистрирован
df_registered = dff.query("date_register < date").drop(columns='gender').drop(columns='month').drop(columns='os_name')
df_registered.head()

,date,event,purchase_sum,device_id,city,utm_source,date_register
16,2020-01-08,app_start,NaN,669460,Moscow,-,2020-01-07
17,2020-01-08,choose_item,NaN,669460,Moscow,-,2020-01-07
18,2020-01-08,search,NaN,669460,Moscow,-,2020-01-07
19,2020-01-08,tap_basket,NaN,669460,Moscow,-,2020-01-07
20,2020-01-09,app_start,NaN,669460,Moscow,-,2020-01-07


In [22]:
df_registered_2 = df_registered.groupby('date_register', as_index = False).agg({'device_id':'count'}).rename(columns={"date_register": "date"}).rename(columns={"device_id": "register_count"})
df_registered_2.head()

,date,register_count
0,2020-01-01,56631
1,2020-01-02,67742
2,2020-01-03,60599
3,2020-01-04,54005
4,2020-01-05,55468


In [23]:
df_search_count = df_registered.query("event == 'search'").groupby('date', as_index = False) \
      .agg({'device_id':'count'}) \
      .rename(columns={"device_id": "search_count"})
df_search_count.head()

,date,search_count
0,2020-01-02,248
1,2020-01-03,567
2,2020-01-04,810
3,2020-01-05,1269
4,2020-01-06,1339


In [24]:
df_choose_item_count = df_registered.query("event == 'choose_item'").groupby('date', as_index = False) \
      .agg({'device_id':'count'}) \
      .rename(columns={"device_id": "choose_item_count"})
df_choose_item_count.head()

,date,choose_item_count
0,2020-01-02,196
1,2020-01-03,437
2,2020-01-04,612
3,2020-01-05,989
4,2020-01-06,1035


In [25]:
df_tap_basket_count = df_registered.query("event == 'tap_basket'").groupby('date', as_index = False) \
      .agg({'device_id':'count'}) \
      .rename(columns={"device_id": "tap_basket_count"})
df_tap_basket_count.head()

,date,tap_basket_count
0,2020-01-02,138
1,2020-01-03,313
2,2020-01-04,437
3,2020-01-05,741
4,2020-01-06,750


In [26]:
q = df_search_count.merge(df_choose_item_count, how = 'inner', on='date')

In [27]:
q = q.merge(df_tap_basket_count, how = 'inner', on='date')
q.head()

,date,search_count,choose_item_count,tap_basket_count
0,2020-01-02,248,196,138
1,2020-01-03,567,437,313
2,2020-01-04,810,612,437
3,2020-01-05,1269,989,741
4,2020-01-06,1339,1035,750


In [28]:
q = q.merge(df_registered_2, how = 'inner', on='date')
q.head()

,date,search_count,choose_item_count,tap_basket_count,register_count
0,2020-01-02,248,196,138,67742
1,2020-01-03,567,437,313,60599
2,2020-01-04,810,612,437,54005
3,2020-01-05,1269,989,741,55468
4,2020-01-06,1339,1035,750,45046


In [29]:
q['CR_search'] = q.search_count / q.register_count *100
q['CR_choose_item'] = q.choose_item_count / q.register_count * 100
q['tap_basket_item'] = q.tap_basket_count / q.register_count * 100

In [43]:
p = q.search_count.sum()

In [44]:
o = q.choose_item_count.sum()

In [45]:
i = q.tap_basket_count.sum()

In [46]:
u = q.register_count.sum()

In [47]:
p / u * 100

29.3929219826407

In [48]:
o / u * 100

22.165471127643777

In [49]:
i / u * 100

15.316968837822378

In [50]:
df_purchase_2 = df.query("event == 'purchase'").groupby(['device_id', 'utm_source'], as_index=False).agg({'date':'min'}).rename(columns={"date": "date_purchase"})

In [51]:
df_purchase_2 = df_purchase_2.groupby(['utm_source'], as_index=False).agg({'device_id':'count'}).rename(columns={"device_id": "purchase_count"})
df_purchase_2.head()

,utm_source,purchase_count
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362


In [52]:
df_app_start_2 = df.query("event == 'app_start'").groupby(['device_id', 'utm_source'], as_index=False).agg({'date':'min'}).rename(columns={"date": "date_app_start"})

In [53]:
df_app_start_2 = df_app_start_2.groupby(['utm_source'], as_index=False).agg({'device_id':'count'}).rename(columns={"device_id": "app_start_count"})
df_app_start_2.head()

,utm_source,app_start_count
0,-,52268
1,facebook_ads,25957
2,google_ads,38092
3,instagram_ads,31045
4,referal,15924


In [54]:
df_channel = df_purchase_2.merge(df_app_start_2, how = 'left', on='utm_source')

In [55]:
df_channel['CR'] = df_channel.purchase_count / df_channel.app_start_count *100

In [56]:
df_channel.sort_values('CR')

,utm_source,purchase_count,app_start_count,CR
6,yandex-direct,12028,40707,29.547744
2,google_ads,11339,38092,29.767405
0,-,16598,52268,31.755567
3,instagram_ads,10762,31045,34.665808
1,facebook_ads,9017,25957,34.738221
5,vk_ads,12364,34079,36.280407
4,referal,6362,15924,39.952273


In [57]:
m = df.query("event == 'purchase'") \
.groupby(['device_id'], as_index=False) \
.agg({'date':'min'}) \
.rename(columns={"date": "date_purchase"})
m

,device_id,date_purchase
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [58]:
df_ff = df.merge(m, how = 'inner', on='device_id')
df_ff.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,date_purchase
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1,2020-01-07
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,1,2020-01-07
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,1,2020-01-07
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,1,2020-01-07
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,1,2020-01-07


In [59]:
df_ff.query("event == 'purchase' and date_purchase == date") \
.groupby('utm_source', as_index=False) \
.agg({'purchase_sum': 'median'}) \
.sort_values('purchase_sum')

,utm_source,purchase_sum
1,facebook_ads,389.0
2,google_ads,390.5
6,yandex-direct,392.5
5,vk_ads,393.0
3,instagram_ads,393.5
4,referal,395.5
0,-,398.5


In [61]:
fffff = df.groupby('utm_source', as_index=False) \
.agg({'purchase_sum': 'sum'})
#добавить столбец с маркетингом
fffff

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [62]:
fffff["marketing"] = [0, 8590498, 10534878, 8561626, 0, 9553531, 10491707]
fffff

,utm_source,purchase_sum,marketing
0,-,21449749.5,0
1,facebook_ads,12249901.0,8590498
2,google_ads,12868276.0,10534878
3,instagram_ads,14546969.0,8561626
4,referal,8837044.5,0
5,vk_ads,16389652.5,9553531
6,yandex-direct,13915368.0,10491707


In [63]:
fffff['ROMI'] = (fffff.purchase_sum - fffff.marketing) / fffff.marketing

In [65]:
fffff.sort_values('ROMI')

,utm_source,purchase_sum,marketing,ROMI
2,google_ads,12868276.0,10534878,0.221493
6,yandex-direct,13915368.0,10491707,0.326321
1,facebook_ads,12249901.0,8590498,0.425983
3,instagram_ads,14546969.0,8561626,0.699090
5,vk_ads,16389652.5,9553531,0.715560
0,-,21449749.5,0,inf
4,referal,8837044.5,0,inf
